# Ollama Webpage summarization
 Before executing the code, ensure that the initial setup outlined in the Readme.md file has been completed.

In [8]:
# Import
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [9]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"   # 11434 port where ollama runs
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"  #Model type

### Test: Verifying Connection with Ollama

In [10]:
# Create a messages list. Has the same format  used on OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }



There is 2 ways of running ollama package:

- Direct HTTP call: you can send HTTP POST requests to the Ollama API
- ollama python package: provides a higher-level interface to interact with the model, abstracting the direct HTTP requests
- Using OpenAI python library to connect to Ollama

This are making web requests locally from my box to my box. And it's connecting to the llama 3.2 model that's being served by llama, and running at localhost:11434


<u> Recommendation:</u>
- If you are primarily using Ollama: Go with ollama Python Package. It’s the cleanest and most idiomatic solution specifically designed for Ollama.
- If you are using both OpenAI and Ollama APIs: Use OpenAI Library (OpenAI) with custom configuration. This provides a unified approach to handling both APIs.
- For advanced or custom needs: Use Direct HTTP Calls (requests.post) for full control and flexibility, especially if you need to customize headers, handle retries, or debug raw API responses.

In [ ]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

In [ ]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

In [ ]:
from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

### Project

In [ ]:
#Prompt Setup:

"""We need to create a structure the APIs understand:
#[ {"role": "system", "content": "system message goes here"},
#  {"role": "user", "content": "user message goes here"} ] """


system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."


def user_prompt_for(website): # A function that writes a User Prompt that asks for summaries of websites:
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt


def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]



In [42]:
# Web Scraping 


class Website: # A class to represent a Webpage, analyize the webpage and extract the information

    def __init__(self, url): # Create this Website object from the given url using the BeautifulSoup librarY
        
        self.url = url

        try: #Handling errors
            response = requests.get(url)
            response.raise_for_status()  # Raises HTTPError if the response code is not 200
            soup = BeautifulSoup(response.content, 'html.parser')
            self.title = soup.title.string if soup.title else "No title found"
            for irrelevant in soup.body(["script", "style", "img", "input"]): #Clean some unseful things
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        except requests.exceptions.RequestException as e:
            print(f"Error fetching the URL: {e}")
            return None


In [ ]:

#Website Class Testing

try:
    law = Website("https://www.law.com/?slreturn=20250125165349")
    print(law.title)
    print(law.text)
    print(user_prompt_for(law))
    messages_for(law) #Test the summary of the webpage

except Exception as e:
    print(f"Error initializing Website object: {e}")



In [ ]:
# Model Execution
import ollama

def summarize(url):  # And now: call the  API, to do the summarization
    website = Website(url) # Web scraping
    try:
        response = ollama.chat(model=MODEL, messages=messages_for(website))  # LLM inference
        return response['message']['content']
    except Exception as e:
        print(f"Error during LLM API call: {e}")
        return None



summarize("https://www.law.com/?slreturn=20250125165349")


In [ ]:

def display_summary(url):# A function to display this nicely in the Jupyter output, using markdown
    summary = summarize(url)
    display(Markdown(summary))

display_summary("https://www.law.com/?slreturn=20250125165349")